In [1]:
# to access functions from root directory
import sys
sys.path.append('/data/ad181/RemoteDir/multilevel_ppo')

In [2]:
%matplotlib notebook
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from tqdm.notebook import trange, tqdm

import gym
from stable_baselines3.ppo import PPO, MlpPolicy
from stable_baselines3.ppo_multi_level import PPO_ML
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env.subproc_vec_multi_level_env import SubprocVecMultiLevelEnv
from stable_baselines3.common.envs.multi_level_ressim_env import MultiLevelRessimEnv
from stable_baselines3.common.logger import configure

from utils.custom_eval_callback import CustomEvalCallback, CustomEvalCallbackParallel
from utils.plot_functions import plot_learning
from utils.env_evaluate_functions import eval_actions

In [3]:
seed=1
case='ppo_2l'
data_dir='./data'
log_dir='./data/'+case

In [4]:
os.makedirs(data_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

In [5]:
with open('../envs_params/env_data_v1/env_train_dict.pkl', 'rb') as input:
    env_ck_dict = pickle.load(input)

In [6]:
# generate dictionaries for env (env_dict_), n_steps (T_ml) and minibatch (M_ml) for `n_level`s
n_levels=2
fine_level = len(env_ck_dict)
env_dict_ = {}
for i,l in enumerate(range(fine_level-n_levels, fine_level)):
    print(i+1,'->',l+1)
    env_dict_[i+1] = env_ck_dict[l+1]

1 -> 3
2 -> 4


In [7]:
for seed in range(2,4):
    if True: 
        print(f'seed {seed}')
        log_dir = './data/'+case+'/seed_'+str(seed)
        os.makedirs(log_dir, exist_ok=True)
        T = {1:140, 2:15} # n_steps
        N = 50 # number of actors
        M = {1:700, 2:75} # minibatch size
        I = 1200 # number of iterations
        K = 20 # number of epochs
        
        log_interval = I/120
        
        fine_level = len(env_dict_)
        
        print('generate callback ...')
        eval_callback = CustomEvalCallback( env_dict_[fine_level], 
                                            best_model_save_path=str(log_dir)+'/best_model', 
                                            n_eval_episodes=1,
                                            log_path=str(log_dir)+'/results_eval', 
                                            eval_freq=log_interval*sum(T.values()) )
        
        print('vectorize environment ...')
        
        # generate PPO_ML parameters for MLMC analysis. 
        env_dict = {}
        n_steps_dict = {}
        batch_size_dict = {}
        for env, level in zip(env_dict_.values(), env_dict_.keys()):
            print(f"vectorize env level {level}")
            env_dict[level] = make_vec_env( MultiLevelRessimEnv, 
                                    n_envs=N, 
                                    seed=seed, 
                                    env_kwargs= {"ressim_params":env.ressim_params, "level":env.level}, 
                                    vec_env_cls=SubprocVecMultiLevelEnv )
            n_steps_dict[level] = T[level]
            batch_size_dict[level] = M[level]
        
        print(env_dict_[level].observation_space)
        print('model definition ..')
        model = PPO_ML(policy=MlpPolicy,
                           env=env_dict,
                           learning_rate = 1e-6,
                           n_steps = n_steps_dict,
                           batch_size = batch_size_dict,
                           n_epochs = K,
                           clip_range = 0.1,
                           ent_coef = 0.001,
                           vf_coef = 0.5,
                           policy_kwargs = dict(net_arch=[70,70,50], log_std_init=-2.9),
                           verbose = 1,
                           seed = seed,
                           target_kl = 0.05,
                           device = "auto")
        # set logger for the model
        new_logger = configure(log_dir)
        model.set_logger(new_logger)
        print('policy learning ..')
        model.learn(total_timesteps=N*sum(T.values())*I, callback=eval_callback)
        model.save(log_dir+'/PPO', exclude=['env_dict'])
        del model
        for level in env_dict.keys():
            env_dict[level].close()


seed 2
generate callback ...
vectorize environment ...
vectorize env level 1
vectorize env level 2
Box(-1.0, 1.0, (35,), float64)
model definition ..
Using cuda device
Logging to ./data/ppo_2l/seed_2
policy learning ..


/data/ad181/RemoteDir/multilevel_ppo/utils/custom_eval_callback.py:146: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.subproc_vec_multi_level_env.SubprocVecMultiLevelEnv object at 0x7fdd8418d250> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x7fdd841c5450>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5        |
|    ep_rew_mean     | 0.687    |
| time/              |          |
|    fps             | 121      |
|    iterations      | 1        |
|    time_elapsed    | 63       |
|    total_timesteps | 7750     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.674        |
| time/                   |              |
|    fps                  | 135          |
|    iterations           | 2            |
|    time_elapsed         | 114          |
|    total_timesteps      | 15500        |
| train/                  |              |
|    approx_kl            | 0.0011907865 |
|    clip_fraction        | 0.0113       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.1        |
|    explained_variance   | -3.23        |
|    learning_r

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):


Eval num_timesteps=77500, episode_reward=0.69 +/- 0.00
Episode length: 5.00 +/- 0.00
New best mean reward!
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 5            |
|    mean_reward          | 0.692        |
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.686        |
| time/                   |              |
|    fps                  | 146          |
|    iterations           | 10           |
|    time_elapsed         | 530          |
|    total_timesteps      | 77500        |
| train/                  |              |
|    approx_kl            | 0.0012481255 |
|    clip_fraction        | 0.0147       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.1        |
|    explained_variance   | -1.82        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.098        |
|    n_updates            | 180  

...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.865        |
| time/                   |              |
|    fps                  | 160          |
|    iterations           | 1181         |
|    time_elapsed         | 56955        |
|    total_timesteps      | 9152750      |
| train/                  |              |
|    approx_kl            | 0.0014482967 |
|    clip_fraction        | 0.0375       |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.5        |
|    explained_variance   | 0.977        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0276       |
|    n_updates            | 23600        |
|    policy_gradient_loss | -0.00146     |
|    std                  | 0.0541       |
|    value_loss           | 0.00166      |
------------------------------------------
------------------------------------------
| rollout/ 

Eval num_timesteps=9222500, episode_reward=0.87 +/- 0.00
Episode length: 5.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 5            |
|    mean_reward          | 0.87         |
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.865        |
| time/                   |              |
|    fps                  | 160          |
|    iterations           | 1190         |
|    time_elapsed         | 57389        |
|    total_timesteps      | 9222500      |
| train/                  |              |
|    approx_kl            | 0.0015363577 |
|    clip_fraction        | 0.032        |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.5        |
|    explained_variance   | 0.977        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0263       |
|    n_updates            | 23780        |
|    polic

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5            |
|    ep_rew_mean          | 0.865        |
| time/                   |              |
|    fps                  | 160          |
|    iterations           | 1199         |
|    time_elapsed         | 57800        |
|    total_timesteps      | 9292250      |
| train/                  |              |
|    approx_kl            | 0.0012927786 |
|    clip_fraction        | 0.042        |
|    clip_range           | 0.1          |
|    entropy_loss         | -31.5        |
|    explained_variance   | 0.977        |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0263       |
|    n_updates            | 23960        |
|    policy_gradient_loss | -0.00279     |
|    std                  | 0.0541       |
|    value_loss           | 0.00174      |
------------------------------------------
Eval num_timesteps=9300000, episode_reward=0.87 +/- 0.

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p

/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p_1)-1):
/data/ad181/RemoteDir/multilevel_ppo/stable_baselines_mlppo/stable_baselines3/common/envs/multi_level_model/level_mapping_functions.py:51: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  for j in range(len(p